# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20901998


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:06<01:29,  3.09s/it]

Rendering models:  10%|▉         | 3/31 [00:08<01:16,  2.73s/it]

Rendering models:  13%|█▎        | 4/31 [00:09<00:59,  2.20s/it]

Rendering models:  16%|█▌        | 5/31 [00:10<00:47,  1.83s/it]

...

Rendering models:  81%|████████  | 25/31 [00:18<00:03,  1.71it/s]

Rendering models:  87%|████████▋ | 27/31 [00:19<00:01,  2.00it/s]

Rendering models:  97%|█████████▋| 30/31 [00:19<00:00,  2.61it/s]

Rendering models: 100%|██████████| 31/31 [00:20<00:00,  2.49it/s]

equidad1                              0.075671
CTidwell3                             0.000213
not-logged-in-20487e5808245554c355    0.000406
ElisabethB                            0.000260
Jes8x                                 0.000981
not-logged-in-b2ab6eb00573e0111cbd    0.002813
not-logged-in-a519260ebf4b014e9045    0.014346
TheoBe                                0.000131
not-logged-in-bc7512d19c1732163dcb    0.012724
kayleebug2017                         0.001443
not-logged-in-4ddf669750977b537491    0.000737
ericmalani                            0.000541
not-logged-in-7db893c424d4c781f559    0.001073
Saharisunshine                        0.000699
pangeli5                              0.000241
Mr_DMan                               0.001111
Lavadude                              0.011673
TateNachampassak                      0.003009
jthrasher                             0.000570
Peyton.Harvey                         0.527648
Andruha_T                             0.001723
jamiwilliams 

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())